<a href="https://colab.research.google.com/github/hoa92ng/Homework/blob/main/Welcome_To_Colaboratory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
Hình~\ref{fig:c2_neural_network_general} minh họa mạng Nơ-ron tổng quát, bao gồm ba tầng chính như sau: Tầng đầu vào (Input Layer), tầng ẩn (Hidden layers) và tầng đầu ra. Kích thước mạng càng sâu thì càng có nhiều lớp tầng ẩn. Mỗi hình tròn tượng trưng cho một Node, các Node này được liên kết đầy đủ tới tất cả các Node ở tầng trước với các hệ số $W$ riêng. Để tăng thêm sự linh hoạt của hàm tổng, mỗi Node sẽ có thêm một hệ số bias $b$. Như có đề cập ở phần trước, tại mỗi Node sẽ diễn ra hai quá trình tính hàm tổng và hàm kích hoạt. Cũng cần lưu ý, đối với các Node trong tầng ẩn thì đầu ra của Node ở tầng trước là đầu vào của Node ở tầng sau, dữ liệu đầu vào được coi là Layer 0 (Input Layer). Ngoài ra chỉ tồn tại các liên kết từ lớp $i$ tới lớp $i + 1$ và không tồn tại liên kết theo chiều ngược lại.

Trong quá trình huấn luyện mạng Nơ-ron sẽ diễn ra hai quá trình lan truyền thẳng (Feedforward) và lan truyền ngược (Backpropagation). Trong quá trình lan truyền thằng giá trí của mỗi Node sẽ được cập nhật thông qua giá trị đầu vào $X$, bộ trọng số $W$, hàm tổng $Z$ và hàm kích hoạt $\sigma$. Còn đối với quá trình lan truyền ngược bộ giá trị trọng số $W$ sẽ được cập nhật thông qua các thuật toán như Gradient Descent và đạo hàm chuỗi.
\begin{enumerate}[\bfseries a)]
	\item {Hoạt động của mạng truyền thẳng}\par
	Mạng truyền thẳng nhiều lớp (Feed forward neural network) là mạng có nhiều lớp ngoài lớp đầu vào và lớp đầu ra. Nó hoạt động theo nguyên tắc kết quả đầu ra của lớp $i$ sẽ là đầu vào của lớp $i + 1$ (trừ lớp đầu ra). Mỗi một lớp có bộ trọng số và vectơ Bias riêng. Các bước cơ bản của quá trình lan truyền thẳng như sau: Tính hàm tổng (\eqref{eq:sum_z}), tính hàm kích hoạt (\eqref{eq:activation_a}).
	\begin{equation*}\footnote{https://machinelearningcoban.com/2017/02/24/mlp/}
		\begin{aligned}
			\textbf{$W$}^(l) \in \mathbb{R}^{d^(l-1)xd^(l)}
			\textbf{$b$}^(l) \in \mathbb{R}^{d^(l)x1}
			z_j^(l) = w_j^{(l)T}a^(l-1) + b_j^(l)
			\textbf{$z$}^(l) =\textbf{$W$}^(l)T\textbf{$a$}^(l-1) + \textbf{$b$}(l)
		\end{aligned}
		\label{eq:sum_z}
		\begin{aligned_2}
			\textbf{$a$^}^(l) = \sigma(\textbf{$z$}^(l)
		\end{aligned_2}
		\label{eq:activation_a}
	\end{equation*}
\end{enumerate}
\begin{enumerate}[\bfseries b)]
	\item {Quá trình lan truyền ngược (Backpropagation)}\par
	Năm 2012, tại cuộc thi ILSVRC,
\end{enumerate}
\begin{figure}[h!]
	\begin{center}
		\includegraphics[width=0.65\linewidth]{images/c2/multi_layers.png}
		\caption{Mạng Nơ-ron tổng quát.\protect\footnotemark}
		\label{fig:c2_neural_network_general}
	\end{center}
\end{figure}
\footnotetext{https://machinelearningcoban.com/2017/02/24/mlp/}

\subsection{Mạng tích chập (Convolutional Neural Network)}